1. Import Llama3
2. Fetch data
3. Initialize few shot examples
4. convert to embeddings
5. few custom tools
6. Prompt
7. Agent with ReAct
8. Agent Executer

Import Llama3

In [1]:
from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model = "llama3.2:1b")

C:\Users\Auxiliar\AppData\Local\Temp\ipykernel_10772\3674769599.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "llama3.2:1b")


In [3]:
llm.invoke("Hi")

'Hello. Is there something I can help you with or would you like to chat?'

Get the Data

In [4]:
# from langchain_community.utilities import SQLDatabase
# db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)

from langchain_community.utilities import SQLDatabase
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Parámetros de conexión a MySQL
user = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
host = os.getenv("HOST")  # O el host donde esté tu base de datos
port = os.getenv("PORT")  # El puerto estándar de MySQL
database = os.getenv("DATABASE")

# Crear la URI de conexión para MySQL
uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"

# Conectar a la base de datos MySQL
db = SQLDatabase.from_uri(uri, sample_rows_in_table_info=3)


In [5]:
print(db.table_info)


CREATE TABLE `AMI_CLIENTES` (
	`M_CLAVE_PRIMARIA` INTEGER NOT NULL AUTO_INCREMENT, 
	`M_NOMBRE_ABONADO` VARCHAR(255), 
	`M_DIRECC_ABONADO` VARCHAR(255), 
	`M_CONTADOR_ACTIVO` VARCHAR(50), 
	`M_MULTIP_ACTIVO` VARCHAR(50), 
	`LAT` DECIMAL(20, 6), 
	`LON` DECIMAL(20, 6), 
	`COORD_X` DECIMAL(20, 6), 
	`COORD_Y` DECIMAL(20, 6), 
	`CIRCUITO` VARCHAR(50), 
	`SUBESTACION` VARCHAR(50), 
	PRIMARY KEY (`M_CLAVE_PRIMARIA`)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from AMI_CLIENTES table:
M_CLAVE_PRIMARIA	M_NOMBRE_ABONADO	M_DIRECC_ABONADO	M_CONTADOR_ACTIVO	M_MULTIP_ACTIVO	LAT	LON	COORD_X	COORD_Y	CIRCUITO	SUBESTACION
30898	DE ROMERO VIRGINIA	HATO DE ENMEDIO         CONT.MARCO MORALE	807010615	1	14.077130	-87.174369	481175.977400	1556263.492000	MFL238	MFL
37811	BUESO FAUSTO N D	COL.EL PERIODISTA ANTES DE LA ESC.PANAMERICANA	56999000	1	14.049414	-87.226019	475597.208800	1553203.000000	TON295	TON
43963	FLORES ANDURAY LENIN EMILIANO	COL EL PERIODISTA       20M ANTES 

In [6]:
prompt = [
  "input: ¿Cuáles son los códigos de alarma que más se repiten en los medidores?",
  "output: SELECT ALARM_CODE, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas GROUP BY ALARM_CODE ORDER BY frecuencia DESC;",
  "input: ¿Qué medidores han generado alarmas en los últimos 7 días?",
  "output: SELECT MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE FECHA >= CURDATE() - INTERVAL 7 DAY;",
  "input: ¿Qué alarmas ha generado el medidor 123456 en el último mes?",
  "output: SELECT ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '123456' AND FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuántas veces se ha registrado la alarma 'Watchdog Error' en todos los medidores?",
  "output: SELECT COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE ALARM_DESC = 'Watchdog Error';",
  "input: ¿Cuántos medidores en el circuito BVI211 presentan la alarma 'Terminal Cover Opened'?",
  "output: SELECT MEDIDOR, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE CLAVE IN (SELECT u.CLAVE_CATALOGO FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AND ALARM_DESC = 'Terminal Cover Opened' GROUP BY MEDIDOR;",
  "input: ¿Con qué frecuencia se ha registrado cada alarma en el último mes?",
  "output: SELECT ALARM_DESC, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY ALARM_DESC;",
  "input: ¿Con qué frecuencia se ha registrado cada alarma en el circuito BVI211 en el último mes?",
  "output: SELECT a.ALARM_DESC, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.ALARM_DESC;",
  "input: ¿Cuantas veces se ha registrado la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes?",
  "output: SELECT COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Con qué frecuencia se ha registrado la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes y cuántos medidores la han presentado?",
  "output: SELECT COUNT(*) AS frecuencia, COUNT(DISTINCT a.MEDIDOR) AS medidores_presentes FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuántas veces han presentado los medidor la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes?",
  "output: SELECT a.MEDIDOR, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito BVI211?",
  "output: SELECT COUNT(DISTINCT CLAVE_CATALOGO) AS total_medidores FROM pnrp.airflow_hexing_universo WHERE CIRCUITO = 'BVI211';",
  "input: ¿Cuántos medidores hay en el circuito BVI211 y cuántas veces ha presentado cada uno la alarma 'Terminal Cover Opened'?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Power Failure' en el último mes?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Power Failure' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Voltage Unbalance' en los últimos 15 días?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Voltage Unbalance' AND a.FECHA >= CURDATE() - INTERVAL 15 DAY GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Overcurrent' en la última semana?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Overcurrent' AND a.FECHA >= CURDATE() - INTERVAL 7 DAY GROUP BY a.MEDIDOR;",
  "input: ¿Cuántas órdenes de servicio hay en el circuito BVI211 en el último mes?",
  "output: SELECT COUNT(*) AS Total_OS FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Qué medidores hay en el circuito BVI211 y cuántas órdenes de servicio tiene cada uno en el último mes?",
  "output: SELECT u.CLAVE_CATALOGO AS Medidor, COUNT(os.OS) AS Total_OS FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH GROUP BY u.CLAVE_CATALOGO;",
  "input: ¿Qué medidores hay en el circuito BVI211 con orden de servicio en el último mes?",
  "output: SELECT os.*, u.CLAVE_CATALOGO AS Medidor FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuáles los tickets para el circuito 'BVI211'?",
  "output: SELECT * FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI211';",
  "input: ¿Quiénes son los analistas que han trabajado en el circuito 'BVI213'?",
  "output: SELECT DISTINCT ANALISTA FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI213';",
  "input: ¿Cuáles son los tickets con no conformidad 'Falta de lectura' en el circuito 'BVI211'?",
  "output: SELECT TICKET, NO_CONFORMIDAD, FECHA_REGISTRO FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI211' AND MARCA = 'HEXING' AND NO_CONFORMIDAD = 'Falta de lectura';",
  "input: ¿Cuáles son los registros de análisis en el circuito 'BVI214' entre el 01-01-2024 y el 31-01-2024?",
  "output: SELECT * FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI214' AND FECHA_ANALISIS BETWEEN '2024-01-01' AND '2024-01-31';",
  "input: ¿Qué alarmas tiene el medidor 'MEDIDOR123' en el periodo de análisis definido en la bitácora para el circuito 'BVI211'?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = 'MEDIDOR123' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL AND b.CIRCUITO = 'BVI211';",
  "input: ¿Cuáles son las alarmas registradas en el circuito 'BVI212' para el medidor 'MEDIDOR456' durante el periodo definido en la bitácora?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = 'MEDIDOR456' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL AND b.CIRCUITO = 'BVI212';",
  "input: ¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = '2023109007594' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL;",
  "input: ¿Cuáles son las últimas alarmas del medidor 12345?",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '12345' ORDER BY FECHA DESC LIMIT 20;",
  "input: Dame las órdenes de servicio del cliente 54321",
  "output: SELECT OS, CLAVE, ESTADO, CATEGORIA, DESCRIPCION_OS, FECHA_GENERADA, FECHA_EJECUCION FROM pnrp.airflow_hexing_os WHERE CLAVE = '54321';",
  "input: Borra todas las entradas de la tabla bitacora_ac",
  "output: SELECT 'No puedo ejecutar esa acción' as ERROR;",
  "input: Actualiza todas las entradas de la tabla airflow_hexing_alarmas a valores null",
  "output: SELECT 'No puedo ejecutar esa acción' as ERROR;",
  "input: ¿Qué alarmas tiene el medidor 67890 y dame solo los primeros 5?",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '67890' ORDER BY FECHA DESC LIMIT 5;",
  "input: Dame todas las lecturas de medidores que empezaron a fallar",
  "output: SELECT 'No puedo responder a esa pregunta' as ERROR;",
  "input: Dame las últimas 10 lecturas del medidor 98765",
  "output: SELECT CLAVE, MEDIDOR, FECHA, LECTURA FROM pnrp.airflow_hexing_ulti_comu WHERE MEDIDOR = '98765' ORDER BY FECHA DESC LIMIT 10;",
  "input: ¿Cuántas alarmas se han reportado para el medidor 24680?",
  "output: SELECT COUNT(*) FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '24680';",
  "input: Dame las órdenes de servicio para el cliente con clave 123456",
  "output: SELECT OS, CLAVE, ESTADO, CATEGORIA, DESCRIPCION_OS, FECHA_GENERADA, FECHA_EJECUCION FROM pnrp.airflow_hexing_os WHERE CLAVE = '123456';",
  "input: Muéstrame las alarmas de los últimos 5 días para el medidor 1234",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '12345' AND FECHA >= DATE_SUB(CURDATE(), INTERVAL 5 DAY) ORDER BY FECHA DESC LIMIT 20;",
  "input: Quiero ver el número de ticket del cliente con clave 654321, una explicación",
  "output: SELECT TICKET, CLAVE, NO_CONFORMIDAD, REQUIERE_OS, COMENTARIO_ANALISTA, ES_RECURRENTE, ANOMALIA, CIRCUITO, ESTADO FROM pnrp.bitacora_ac WHERE CLAVE = '654321';", 
]

# Inicializar el array de JSON
queries = []

# Iterar sobre los elementos del array de dos en dos (pares de input y output)
for i in range(0, len(prompt), 2):
    input_str = prompt[i].replace("input: ", "").strip()  # Limpiar el texto 'input:'
    output_str = prompt[i + 1].replace("output: ", "").strip()  # Limpiar el texto 'output:'
    
    # Crear el objeto JSON
    query = {
        "input": input_str,
        "query": output_str
    }
    
    # Añadir al array
    queries.append(query)

# Mostrar el resultado
import json
print(json.dumps(queries, indent=2))

[
  {
    "input": "\u00bfCu\u00e1les son los c\u00f3digos de alarma que m\u00e1s se repiten en los medidores?",
    "query": "SELECT ALARM_CODE, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas GROUP BY ALARM_CODE ORDER BY frecuencia DESC;"
  },
  {
    "input": "\u00bfQu\u00e9 medidores han generado alarmas en los \u00faltimos 7 d\u00edas?",
    "query": "SELECT MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE FECHA >= CURDATE() - INTERVAL 7 DAY;"
  },
  {
    "input": "\u00bfQu\u00e9 alarmas ha generado el medidor 123456 en el \u00faltimo mes?",
    "query": "SELECT ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '123456' AND FECHA >= CURDATE() - INTERVAL 1 MONTH;"
  },
  {
    "input": "\u00bfCu\u00e1ntas veces se ha registrado la alarma 'Watchdog Error' en todos los medidores?",
    "query": "SELECT COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE ALARM_DESC = 'Watchdog Error';"
  },
  {
    "input": "

Initialize Few Shots

In [7]:
# examples = [
#     {   "input": "List all artists.", 
#         "query": "SELECT * FROM Artist;"},
#     {
#         "input": "Find all albums for the artist 'AC/DC'.",
#         "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
#     },
#     {
#         "input": "List all tracks in the 'Rock' genre.",
#         "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
#     },
#     {
#         "input": "Find the total duration of all tracks.",
#         "query": "SELECT SUM(Milliseconds) FROM Track;",
#     },
#     {
#         "input": "List all customers from Canada.",
#         "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
#     },
#     {
#         "input": "How many tracks are there in the album with ID 5?",
#         "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
#     },
#     {
#         "input": "Find the total number of Albums.",
#         "query": "SELECT COUNT(DISTINT(AlbumId)) FROM Invoice;",
#     },
#     {
#         "input": "List all tracks that are longer than 5 minutes.",
#         "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
#     },
#     {
#         "input": "Who are the top 5 customers by total purchase?",
#         "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
#     },
#     {
#         "input": "How many employees are there",
#         "query": 'SELECT COUNT(*) FROM "Employee"',
#     },
# ]

examples = queries
print(len(examples))
# primer ejemplo
print(examples[0])

36
{'input': '¿Cuáles son los códigos de alarma que más se repiten en los medidores?', 'query': 'SELECT ALARM_CODE, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas GROUP BY ALARM_CODE ORDER BY frecuencia DESC;'}


Few Shots to Embeddings

In [8]:
from langchain_community.embeddings import OllamaEmbeddings

In [9]:
embeddings = (
    OllamaEmbeddings(model = "llama3.2:1b")
)

C:\Users\Auxiliar\AppData\Local\Temp\ipykernel_10772\3565120015.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(model = "llama3.2:1b")


In [10]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=3,
    input_keys=["input"],
    )

In [11]:
matched_queries = example_selector.vectorstore.search("¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI211'?", search_type = "mmr")
print(matched_queries)

[Document(metadata={'input': "¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?", 'query': "SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = '2023109007594' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL;"}, page_content="¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?"), Document(metadata={'input': "¿Cuántos medidores hay en el circuito BVI211 y cuántas veces ha presentado cada uno la alarma 'Terminal Cover Opened'?", 'query': "SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' GROUP BY a.MEDIDOR;"}, page_conte

[Document(metadata={'input': "¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?", 'query': "SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = '2023109007594' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL;"}, page_content="¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?"), Document(metadata={'input': "¿Cuántos medidores hay en el circuito BVI211 y cuántas veces ha presentado cada uno la alarma 'Terminal Cover Opened'?", 'query': "SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' GROUP BY a.MEDIDOR;"}, page_content="¿Cuántos medidores hay en el circuito BVI211 y cuántas veces ha presentado cada uno la alarma 'Terminal Cover Opened'?"), Document(metadata={'input': '¿Qué medidores hay en el circuito BVI211 con orden de servicio en el último mes?', 'query': "SELECT os.*, u.CLAVE_CATALOGO AS Medidor FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH;"}, page_content='¿Qué medidores hay en el circuito BVI211 con orden de servicio en el último mes?'), Document(metadata={'input': "¿Cuántos medidores en el circuito BVI211 presentan la alarma 'Terminal Cover Opened'?", 'query': "SELECT MEDIDOR, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE CLAVE IN (SELECT u.CLAVE_CATALOGO FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AND ALARM_DESC = 'Terminal Cover Opened' GROUP BY MEDIDOR;"}, page_content="¿Cuántos medidores en el circuito BVI211 presentan la alarma 'Terminal Cover Opened'?")]


In [12]:
for doc in matched_queries:
    print(doc.page_content)

¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?
¿Cuántos medidores hay en el circuito BVI211 y cuántas veces ha presentado cada uno la alarma 'Terminal Cover Opened'?
¿Qué medidores hay en el circuito BVI211 con orden de servicio en el último mes?
¿Cuántos medidores en el circuito BVI211 presentan la alarma 'Terminal Cover Opened'?


Custom Tools

In [13]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, InfoSQLDatabaseTool, ListSQLDatabaseTool, QuerySQLCheckerTool

sql_db_query =  QuerySQLDataBaseTool(db = db)
sql_db_schema =  InfoSQLDatabaseTool(db = db)
sql_db_list_tables =  ListSQLDatabaseTool(db = db)
sql_db_query_checker = QuerySQLCheckerTool(db = db, llm = llm)

tools = [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker]

for tool in tools:
    print(tool.name + " - " + tool.description.strip() + "\n")

sql_db_query - Execute a SQL query against the database and get back the result..
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.

sql_db_schema - Get the schema and sample rows for the specified SQL tables.

sql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker - Use this tool to double check if your query is correct before executing it.
    Always use this tool before executing a query with sql_db_query!



Prompts

In [14]:
system_prefix = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Here are some examples of user inputs and their corresponding SQL queries:

"""

In [15]:
suffix = """
Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [16]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate

In [17]:
dynamic_few_shot_prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input"],
    prefix=system_prefix,
    suffix=suffix
)

In [18]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=dynamic_few_shot_prompt_template),
    ]
)

In [19]:
prompt_val = full_prompt.invoke(
    {
        "input": "¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?",
        "tool_names" : [tool.name for tool in tools],
        "tools" : [tool.name + " - " + tool.description.strip() for tool in tools],
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

output_string = prompt_val.to_string()

# Extraer la consulta SQL utilizando una búsqueda de texto
# Busca la línea que comienza con "SQL query:" y extrae el texto siguiente.
sql_query = ""
lines = output_string.split('\n')
for line in lines:
    if line.startswith("SQL query:"):
        sql_query = line.split("SQL query:")[1].strip()  # Obtener la consulta SQL
        break

# Imprimir solo la consulta SQL
print(sql_query)

System: 
Answer the following questions as best you can. You have access to the following tools:

['sql_db_query - Execute a SQL query against the database and get back the result..\n    If the query is not correct, an error message will be returned.\n    If an error is returned, rewrite the query, check the query, and try again.', 'sql_db_schema - Get the schema and sample rows for the specified SQL tables.', 'sql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.', 'sql_db_query_checker - Use this tool to double check if your query is correct before executing it.\n    Always use this tool before executing a query with sql_db_query!']

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']]
Action Input: the input to the action
Observation: the

Create Agent

In [22]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(llm, tools, full_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
import langchain
langchain.debug = True

In [23]:
agent_executor.invoke({"input": "¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?", "query": ""})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?",
  "query": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agen

{'input': "¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?",
 'query': '',
 'output': 'Agent stopped due to iteration limit or time limit.'}